## Part 1: Preprocessing

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [5]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [6]:

dept_attrition_df = attrition_df.copy()

# Display a sample of y
y_df=pd.DataFrame(dept_attrition_df[["Attrition","Department"]])

y_df

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [7]:
attrition_df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [8]:
# Create y_df with the Attrition and Department columns



In [9]:
X_df = pd.DataFrame(attrition_df[['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime',
       'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion',
       'NumCompaniesWorked']])

X_df                    

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
0,2,41,1,4,Yes,0,1,6,0,8
1,1,49,8,2,No,1,3,10,1,1
2,2,37,2,3,Yes,0,3,0,0,6
3,4,33,3,3,Yes,0,3,8,3,1
4,1,27,2,2,No,1,3,2,2,9
...,...,...,...,...,...,...,...,...,...,...
1465,2,36,23,4,No,1,3,5,0,4
1466,1,39,6,1,No,1,3,7,1,4
1467,3,27,4,2,Yes,1,3,6,0,1
1468,3,49,2,2,No,0,2,9,0,2


In [10]:
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [11]:
def status_change_int(xData):
    if (xData['OverTime'] == 'Yes'):
        return 1
    return 0
X_df['OverTime']=X_df.apply(status_change_int,axis=1)
X_df.dtypes

Education                  int64
Age                        int64
DistanceFromHome           int64
JobSatisfaction            int64
OverTime                   int64
StockOptionLevel           int64
WorkLifeBalance            int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
NumCompaniesWorked         int64
dtype: object

In [124]:
X_df

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
0,2,41,1,4,1,0,1,6,0,8
1,1,49,8,2,0,1,3,10,1,1
2,2,37,2,3,1,0,3,0,0,6
3,4,33,3,3,1,0,3,8,3,1
4,1,27,2,2,0,1,3,2,2,9
...,...,...,...,...,...,...,...,...,...,...
1465,2,36,23,4,0,1,3,5,0,4
1466,1,39,6,1,0,1,3,7,1,4
1467,3,27,4,2,1,1,3,6,0,1
1468,3,49,2,2,0,0,2,9,0,2


In [4]:
# Create a list of at least 10 column names to use as X data



# Create X_df using your selected columns


# Show the data types for X_df



Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [12]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)


In [133]:
X_test

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
501,3,30,3,3,0,1,3,1,0,1
848,4,35,4,4,0,1,4,2,2,1
735,3,48,6,3,0,0,3,2,2,2
36,2,50,3,3,1,0,3,3,0,1
410,3,30,2,4,0,0,3,5,0,2
...,...,...,...,...,...,...,...,...,...,...
759,4,45,24,2,0,0,3,6,0,1
814,3,40,14,3,0,0,4,20,4,1
348,5,42,23,4,0,0,3,1,0,2
211,1,30,1,3,0,0,3,11,5,1


In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary


No     788
Yes    314
Name: OverTime, dtype: int64

In [17]:
y_df["Department"].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [18]:
y_df["Attrition"].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

In [57]:
X_df["OverTime"].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [39]:
def status_change_int_ydata(xData):
    if (xData['Department'] == 'Sales'):
        return 1
    if (xData['Department'] == 'Human Resources'):
        return 2
    else:
        return 3
    return 0
y_df['Department']=y_df.apply(status_change_int_ydata,axis=1)
y_df.dtypes

Attrition     object
Department     int64
dtype: object

In [40]:
def status_change_int_ydata(xData):
    if (xData['Attrition'] == 'Yes'):
        return 1
    else:
        return 0
    return 0
y_df['Attrition']=y_df.apply(status_change_int_ydata,axis=1)
y_df.dtypes

Attrition     int64
Department    int64
dtype: object

In [67]:
y_train.dtypes

Attrition     int64
Department    int64
dtype: object

In [128]:
def status_change_int_ydata(xData):
    if (xData['Attrition'] == 'Yes'):
        return 1
    else:
        return 0
    return 0
y_train['Attrition']=y_train.apply(status_change_int_ydata,axis=1)
y_train.dtypes

Attrition      int64
Department    object
dtype: object

In [129]:
def status_change_int_ydata(xData):
    if (xData['Department'] == 'Sales'):
        return 1
    if (xData['Department'] == 'Human Resources'):
        return 2
    else:
        return 3
    return 0
y_train['Department']=y_train.apply(status_change_int_ydata,axis=1)
y_train.dtypes

Attrition     int64
Department    int64
dtype: object

In [71]:
def status_change_int_ydata(xData):
    if (xData['Attrition'] == 'Yes'):
        return 1
    else:
        return 3
    return 0
y_df['Attrition']=y_df.apply(status_change_int_ydata,axis=1)
y_df.dtypes

Attrition      int64
Department    object
dtype: object

In [101]:
def status_change_int_ydata(xData):
    if (xData['Department'] == 'Sales'):
        return 1
    if (xData['Department'] == 'Human Resources'):
        return 2
    else:
        return 3
    return 0
y_df['Department']=y_df.apply(status_change_int_ydata,axis=1)
y_df.dtypes

Attrition            object
Department            int64
attrition_encoded     int32
dtype: object

In [15]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
#y_scaler = StandardScaler().fit(y_train)

In [16]:
#from sklearn.preprocessing import StandardScaler
#X_scaler = StandardScaler().fit(X_train)
#y_scaler = StandardScaler().fit(y_train)

X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)
#y_train_scaled = y_scaler.transform(y_train)
#y_test_scaled = y_scaler.transform(y_test)

In [90]:
def status_change_int(xData):
    if (xData['OverTime'] == 'Yes'):
        return 1
    return 0
X_test['OverTime']=X_test.apply(status_change_int,axis=1)
X_test.dtypes

Education                  int64
Age                        int64
DistanceFromHome           int64
JobSatisfaction            int64
OverTime                   int64
StockOptionLevel           int64
WorkLifeBalance            int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
NumCompaniesWorked         int64
dtype: object

In [66]:
def status_change_int_ydata(xData):
    if (xData['Attrition'] == 'Yes'):
        return 1
    else:
        return 3
    return 0
y_test['Attrition']=y_test.apply(status_change_int_ydata,axis=1)
y_test.dtypes

Attrition      int64
Department    object
dtype: object

In [67]:
def status_change_int_ydata(xData):
    if (xData['Department'] == 'Sales'):
        return 1
    if (xData['Department'] == 'Human Resources'):
        return 2
    else:
        return 3
    return 0
y_test['Department']=y_test.apply(status_change_int_ydata,axis=1)
y_test.dtypes

Attrition     int64
Department    int64
dtype: object

In [42]:
X_test

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
103,4,34,6,3,0,0,3,16,2,1
423,4,30,22,1,0,0,3,9,7,0
491,5,43,9,3,1,1,3,8,4,3
1275,3,51,3,3,0,0,3,2,2,2
1323,2,28,1,4,0,1,3,3,2,1
...,...,...,...,...,...,...,...,...,...,...
198,4,38,2,3,0,0,3,1,0,9
193,3,43,7,4,0,3,4,5,2,4
946,4,40,25,2,1,0,3,5,1,2
999,3,42,10,1,0,1,3,20,0,0


In [68]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [8]:
# Create a StandardScaler


# Fit the StandardScaler to the training data


# Scale the training and testing data



In [156]:
y_train

,Attrition,Department
5,No,Research & Development
1390,Yes,Research & Development
215,No,Sales
970,No,Sales
65,No,Research & Development
...,...,...
1072,No,Research & Development
1447,No,Sales
847,No,Research & Development
614,Yes,Research & Development


In [17]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Preprocess "Department" column (one-hot encoding)
dept_encoder = OneHotEncoder(sparse_output=False)
dept_encoded = dept_encoder.fit_transform(y_train[['Department']])
dept_columns = dept_encoder.get_feature_names_out(['Department'])
df_dept_encoded = pd.DataFrame(dept_encoded, columns=dept_columns)
df_dept_encoded.values


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [18]:
# Preprocess "attrition" column (label encoding for binary; one-hot encoding for multiple categories)
attrition_encoder = LabelEncoder()
y_train['attrition_encoded'] = attrition_encoder.fit_transform(y_train['Attrition'])

# Concatenate the encoded columns to the original DataFrame
df_processed = pd.concat([y_train, df_dept_encoded], axis=1)

# Drop the original "quality" and "color" columns
df_processed = df_processed.drop(['Department', 'Attrition'], axis=1)

df_processed

,attrition_encoded,Department_Human Resources,Department_Research & Development,Department_Sales
40,0.0,0.0,1.0,0.0
834,0.0,0.0,0.0,1.0
516,0.0,1.0,0.0,0.0
600,0.0,0.0,1.0,0.0
297,0.0,0.0,0.0,1.0
...,...,...,...,...
1090,NaN,0.0,1.0,0.0
1092,NaN,0.0,0.0,1.0
1097,NaN,0.0,0.0,1.0
1098,NaN,0.0,1.0,0.0


In [21]:
y_attrition = df_processed['attrition_encoded']

y_dept = df_processed[['Department_Human Resources', 'Department_Research & Development', 'Department_Sales']]

In [22]:
y_dept

,Department_Human Resources,Department_Research & Development,Department_Sales
40,0.0,1.0,0.0
834,0.0,0.0,1.0
516,1.0,0.0,0.0
600,0.0,1.0,0.0
297,0.0,0.0,1.0
...,...,...,...
1090,0.0,1.0,0.0
1092,0.0,0.0,1.0
1097,0.0,0.0,1.0
1098,0.0,1.0,0.0


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_attrition_train, y_attrition_test, y_dept_train, y_dept_test = train_test_split(X_train_scaled, y_attrition, y_dept)



ValueError: Found input variables with inconsistent numbers of samples: [1102, 1381, 1381]

In [175]:
input_layer = layers.Input(shape=(X_df.shape[1],), name='input_features')

# Shared hidden layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)

In [176]:
dept_output = layers.Dense(32, activation='softmax', name='dept_output')(shared_layer2)

# Branch for color prediction
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(shared_layer2)

In [177]:
model = Model(inputs=input_layer, outputs=[dept_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'dept_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'dept_output': 'accuracy', 'attrition_output': 'accuracy'})

# Display the model summary
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features (InputLayer)   │ (None, 10)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_12 (Dense)              │ (None, 64)                │             704 │ input_features[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_13 (Dense)              │ (None, 32)                │           2,080 │ dense_12[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dept_output (Dense)           │ (None, 3)                 │              99 │ dense_13[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attrition_output (Dense)      │ (None, 1)                 │              33 │ dense_13[0][0]             │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,916 (11.39 KB)

 Trainable params: 2,916 (11.39 KB)

 Non-trainable params: 0 (0.00 B)

In [163]:
X_train

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
5,2,32,2,4,0,0,2,7,3,0
1390,3,28,17,4,0,1,2,4,0,5
215,3,41,6,4,1,0,3,1,0,3
970,3,27,11,4,0,1,3,1,0,8
65,3,55,8,3,1,3,3,5,0,2
...,...,...,...,...,...,...,...,...,...,...
1072,1,29,2,2,0,3,3,3,0,0
1447,4,36,15,4,0,1,2,15,11,1
847,3,34,1,1,0,0,3,13,4,0
614,2,26,5,3,1,1,3,8,1,1


In [179]:

model.fit(
    X_train,
    {'dept_output': y_dept, 'attrition_output': y_attrition},
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 3)

In [180]:
test_results = model.evaluate(X_test, {'dept_output': y_dept_test, 'attrition_output': y_attrition_test})
test_results

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 3)

In [11]:
# Create a OneHotEncoder for the Attrition column


# Fit the encoder to the training data


# Create two new variables by applying the encoder
# to the training and testing data


# Fit_transform the eyes column

# Get the feature names from the encoder

# Create a new DataFrame with the data

# View the first few rows of the data

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [44]:
X_df.shape[1]

10

In [48]:
# Find the number of columns in the X training data


# Create the input layer


# Create at least two shared layers
input_layer = layers.Input(shape=(y_df.shape[1],), name='input_features')

# Shared hidden layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [45]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer

department_output = layers.Dense(3, activation='softmax', name='Department')(shared_layer2)
attrition_output = layers.Dense(3, activation='softmax', name='Attrition')(shared_layer2)

In [46]:
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Display the model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features (InputLayer)   │ (None, 10)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 64)                │             704 │ input_features[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 32)                │           2,080 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Department (Dense)            │ (None, 3)                 │              99 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Attrition (Dense)             │ (None, 3)                 │              99 │ dense_1[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,982 (11.65 KB)

 Trainable params: 2,982 (11.65 KB)

 Non-trainable params: 0 (0.00 B)

In [104]:

model.fit(
    X_train,
    {'department_output': y_df["Department"], 'attrition_output': y_df["Attrition"]},
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/10


AttributeError: 'NoneType' object has no attribute 'items'

In [14]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer
quality_output = layers.Dense(3, activation='softmax', name='quality_output')(shared_layer2)

# Branch for color prediction
color_output = layers.Dense(1, activation='sigmoid', name='color_output')(shared_layer2)


In [15]:
# Create the model


# Compile the model


# Summarize the model

model = Model(inputs=input_layer, outputs=[quality_output, color_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'quality_output': 'categorical_crossentropy', 'color_output': 'binary_crossentropy'},
              metrics={'quality_output': 'accuracy', 'color_output': 'accuracy'})

# Display the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [16]:
# Train the model

model.fit(
    X,
    {'quality_output': y_df, 'color_output': y_color},
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/100
35/35 [==============================] - 14s 25ms/step - loss: 1.4162 - department_output_loss: 0.8374 - attrition_output_loss: 0.5787 - department_output_accuracy: 0.6316 - attrition_output_accuracy: 0.7024
Epoch 2/100
35/35 [==============================] - 1s 27ms/step - loss: 1.2070 - department_output_loss: 0.7661 - attrition_output_loss: 0.4408 - department_output_accuracy: 0.6570 - attrition_output_accuracy: 0.8403
Epoch 3/100
35/35 [==============================] - 0s 13ms/step - loss: 1.1520 - department_output_loss: 0.7539 - attrition_output_loss: 0.3981 - department_output_accuracy: 0.6570 - attrition_output_accuracy: 0.8403
Epoch 4/100
35/35 [==============================] - 0s 14ms/step - loss: 1.1231 - department_output_loss: 0.7452 - attrition_output_loss: 0.3779 - department_output_accuracy: 0.6588 - attrition_output_accuracy: 0.8475
Epoch 5/100
35/35 [==============================] - 0s 12ms/step - loss: 1.1031 - department_output_loss: 0.7342 - attriti

In [17]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {'quality_output': y_quality_test, 'color_output': y_color_test})
test_results

12/12 [==============================] - 0s 3ms/step - loss: 4.1624 - department_output_loss: 3.1143 - attrition_output_loss: 1.0481 - department_output_accuracy: 0.5272 - attrition_output_accuracy: 0.8261


[4.162380218505859,
 3.114327907562256,
 1.0480519533157349,
 0.5271739363670349,
 0.8260869383811951]

In [18]:
# Print the accuracy for both department and attrition


Department predictions accuracy: 0.5271739363670349
Attrition predictions accuracy: 0.8260869383811951


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 